In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [91]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [92]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive
!ls
!free -g

datalab  drive
              total        used        free      shared  buff/cache   available
Mem:             12           1           0           0          10          10
Swap:             0           0           0


In [0]:
# Using Keras functional model
import keras
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Dense, LSTM
# Used for feeding data in Input 
import numpy as np

In [0]:
batch_size =64 # Batchsize for network
epochs = 50 # Number of epochs
latent_dim = 256 # Number of LSTM cells
num_samples = 2000 # Number of samples

In [0]:
input_texts = [] # List of Input(English sentences)
target_texts = [] # List of Target(Hindi sentences)

In [0]:
input_chars = [] # List of unique english characters
target_chars = [] # List of unique hindi characters

In [0]:
file_path = 'drive/projects/nmt-hindi-english/data/eng-hin.txt' # Path to english hindi text file

In [0]:
# reading file into lines
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [0]:
for line in lines[:max(num_samples, len(lines)-1)]:
    input_text, target_text = line.split('\t') # Splitting using tabspaces
    target_text = '\t' + target_text + '\n' # Using \n token to signify EOS 
    input_texts.append(input_text)
    target_texts.append(target_text)
    # Used to get unique characters
    for char in input_text:
        if char not in input_chars:
            input_chars.append(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.append(char)

In [0]:
input_chars = sorted(input_chars)
target_chars = sorted(target_chars) 

In [101]:
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(seq) for seq in input_texts]) # Maximum sequence length for english sentences
max_decoder_seq_length = max([len(seq) for seq in target_texts]) # Maximum sequence length for hindi sentences
max_encoder_seq_length,max_decoder_seq_length

(124, 123)

In [0]:
# Mapping character tokens to indices
input_token_index = dict( [(char, i) for i,char in enumerate(input_chars)])
target_token_index = dict( [(char, i) for i,char in enumerate(target_chars)])

In [0]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype= 'float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')

In [0]:
# One-hot encoding input
for i, input_text in enumerate(input_texts):
    for j, ch in enumerate(input_text):
        encoder_input_data[i,j,input_token_index[ch]] =1

In [0]:
# One-hot encoding output
for i, target_text in enumerate(target_texts):
    for j, ch in enumerate(target_text):
        decoder_input_data[i,j,target_token_index[ch]] =1
        if j>0:
            decoder_target_data[i, j-1, target_token_index[ch]]=1

In [0]:
# Defining Encoder model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [0]:
encoder_states = [state_h, state_c]

In [0]:
# Defining Decoder model
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_state = True, return_sequences = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model = load_model('drive/projects/nmt-hindi-english/models/last_chkpt.h5')

In [0]:
model.compile(optimizer = 'rmsprop', loss= 'categorical_crossentropy')

In [0]:
callbacks_list = [
        keras.callbacks.ReduceLROnPlateau(
            monitor='loss',
            factor=0.1,
            patience=10,
        ),
]

In [162]:
history = model.fit([encoder_input_data,decoder_input_data], decoder_target_data,\
                    batch_size= batch_size, epochs=20, callbacks = callbacks_list)

Epoch 1/20
2867/2867 [==============================] - 37s 13ms/step - loss: 0.9202
Epoch 2/20
2867/2867 [==============================] - 33s 11ms/step - loss: 0.8510
Epoch 3/20
2867/2867 [==============================] - 33s 11ms/step - loss: 0.7713
Epoch 4/20
2867/2867 [==============================] - 31s 11ms/step - loss: 0.7014
Epoch 5/20
2867/2867 [==============================] - 30s 10ms/step - loss: 0.6472
Epoch 6/20
2867/2867 [==============================] - 31s 11ms/step - loss: 0.6103
Epoch 7/20
2112/2867 [=====================>........] - ETA: 8s - loss: 0.6071

2867/2867 [==============================] - 33s 11ms/step - loss: 0.5990
Epoch 8/20
2867/2867 [==============================] - 33s 11ms/step - loss: 0.5633
Epoch 9/20
2867/2867 [==============================] - 33s 12ms/step - loss: 0.5482
Epoch 10/20
2867/2867 [==============================] - 32s 11ms/step - loss: 0.5336
Epoch 11/20
2867/2867 [==============================] - 30s 11ms/step - loss: 0.5221
Epoch 12/20
2867/2867 [==============================] - 30s 10ms/step - loss: 0.5113
Epoch 13/20
2624/2867 [==========================>...] - ETA: 2s - loss: 0.5025

2867/2867 [==============================] - 33s 11ms/step - loss: 0.5012
Epoch 14/20
2867/2867 [==============================] - 33s 12ms/step - loss: 0.4921
Epoch 15/20
2867/2867 [==============================] - 33s 12ms/step - loss: 0.4836
Epoch 16/20
2867/2867 [==============================] - 32s 11ms/step - loss: 0.4830
Epoch 17/20
2867/2867 [==============================] - 30s 11ms/step - loss: 0.4773
Epoch 18/20
2867/2867 [==============================] - 30s 11ms/step - loss: 0.4651
Epoch 19/20
2688/2867 [===========================>..] - ETA: 2s - loss: 0.4595

2867/2867 [==============================] - 33s 11ms/step - loss: 0.4567
Epoch 20/20
2867/2867 [==============================] - 33s 12ms/step - loss: 0.4493


In [163]:
model.save('drive/projects/nmt-hindi-english/models/temp_chkpt.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_12 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_11/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_11/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [0]:
# model = load_model('drive/projects/nmt-hindi-english/models/last_chkpt.h5')

In [0]:
def sample(preds, temperature= 1.0):
    preds = np.reshape(preds,preds.shape[-1])
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds, 1)
    return np.argmax(probas)

In [0]:
# Defining inference model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        next_index = sample(output_tokens,0.5)
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[next_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, next_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
def make_inference(sentence):
    input_data= np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype= 'float32')
    for j, ch in enumerate(sentence):
        input_data[0,j,input_token_index[ch]] =1
    decoded_sentence = decode_sequence(input_data)
    print('Input sentence:', sentence)
    print('Translated sentence:', decoded_sentence)

In [179]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Wow!
Decoded sentence: वह कुम बाल के बार अपने का समझ क्या है।

-
Input sentence: Help!
Decoded sentence: तुम्हें कर दिया।

-
Input sentence: Jump.
Decoded sentence: मुझे अपनी उसकी समी पर गया है।

-
Input sentence: Jump.
Decoded sentence: को को मुझे ज़्दद कर सकते हैं।

-
Input sentence: Jump.
Decoded sentence: उसकी बार कर से में कल समय कर नहीं है।

-
Input sentence: Hello!
Decoded sentence: मुझे अपने कार करने को मुझे किया था।

-
Input sentence: Hello!
Decoded sentence: वह तम अपने साम को मुझे में उसे पाल तुम्हें हैं।

-
Input sentence: Cheers!
Decoded sentence: मुझे अपने अपने को गोड़ कर बार कर रही है।

-
Input sentence: Cheers!
Decoded sentence: उसने मुझे अपनी आद की बार शोन कर दिया।

-
Input sentence: Got it?
Decoded sentence: मैं तुम्हें पर्चा किया था।

